In [9]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
def parse_html_table(html_table):
    data = []
    cur_row = []
    row_names = []
    for ele in html_table:
        stat_name  = ele['data-stat']
        stat_value = ele.string
        new_row = (stat_name == 'player')
        if new_row:
            if cur_row:
                data.append(cur_row)
            cur_row = []
            col_names = []
            cur_row.append(ele['csk']) 
            col_names.append(stat_name)
            continue
        cur_row.append(stat_value)
        col_names.append(stat_name)
    return data, col_names

In [10]:
dfs = []

for year in range(1980,(2021 + 1)):
    url = 'http://www.basketball-reference.com/leagues/NBA_{year}_totals.html'.format(year = year)
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    yr_data, col_names = parse_html_table(soup.findAll('td'))
    df = pd.DataFrame(yr_data, columns = col_names)
    df['yr'] = year
    dfs.append(df)

all_seasons = pd.concat(dfs)
all_seasons.to_csv('data/season_totals.csv')